In [1]:
import numpy as np
import autograd as ag
import pandas as pd
from autograd import numpy as agnp
from autograd import scipy
from scipy.linalg import expm

from sklearn.decomposition import PCA
from multiprocessing import Process, Manager

import time
import plotly.offline as pyo
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
pyo.init_notebook_mode(connected=True)

<h1> Embedded Manifold HMC </h1>

The Geodesic HMC algorithm is a small modification of classic HMC.  The structure of the code is 90% the same as HMC with the modification in that the integration is step is now repeated application of projection and flow steps.

In [2]:
"""
    HMC but now with corrections to sample on an embedded manifold.

    Inputs:
        initial_params : initial parameters
        n_iters        : number of samples
        epsilon        : leapfrog step size
        L              : number of leapfrog steps
        U              : potential energy w.r.t Hausdorff Measure
        grad_U         : gradient of potential energy w.r.t the Hausdorff Measure
        N              : function for the orthogonal projection basis of the problem
        flow           : geodesic flow update for q and p
        print_iter     : how often to print the sample number
"""
def Embedded_Manifold_HMC(initial_params, n_iters, epsilon, L, U, grad_U, flow, projection, print_iter = 1000):

    chain = [initial_params]
    accepts = 0
    rejects = 0

    for it in range(1, n_iters):
        
        q = chain[it - 1].copy()
        p = np.random.normal(0, 1, size = q.shape)
        #p = p - np.dot(np.dot(N(q), N(q).T),p)
        p = projection(p, q)

        # Compute current potential and kinetic energy
        current_U  = U(q)
        current_K = np.sum(p**2) / 2
       
        for i in range(L):
            p = p - epsilon * grad_U(q) / 2
            p = projection(p, q)
            q, p = flow(q, p, epsilon)
            p = p - epsilon * grad_U(q) / 2
            p = projection(p,q)

        proposed_U = U(q)
        proposed_K = np.sum(p**2) / 2

        if np.random.uniform(0,1) < np.exp(current_U - proposed_U + current_K - proposed_K):
            chain.append(q)
            accepts += 1
        else:
            chain.append(chain[it-1])
            rejects += 1

        if it % print_iter == 0:
            print(it)

    print(accepts)
    print(rejects)

    return chain

def Embedded_Manifold_HMC_MT(initial_params, n_iters, epsilon, L, U, grad_U, flow, projection, proc_num, return_dict, print_iter = 10000):
    np.random.seed()
    chain = [initial_params]
    accepts = 0
    rejects = 0

    for it in range(1, n_iters):
        
        q = chain[it - 1].copy()
        p = np.random.normal(0, 1, size = q.shape)
        #p = p - np.dot(np.dot(N(q), N(q).T),p)
        p = projection(p, q)

        # Compute current potential and kinetic energy
        current_U  = U(q)
        current_K = np.sum(p**2) / 2
       
        for i in range(L):
            p = p - epsilon * grad_U(q) / 2
            p = projection(p, q)
            q, p = flow(q, p, epsilon)
            p = p - epsilon * grad_U(q) / 2
            p = projection(p,q)

        proposed_U = U(q)
        proposed_K = np.sum(p**2) / 2

        if np.random.uniform(0,1) < np.exp(current_U - proposed_U + current_K - proposed_K):
            chain.append(q)
            accepts += 1
        else:
            chain.append(chain[it-1])
            rejects += 1

        if it % print_iter == 0:
            print(it)

    return_dict[proc_num] = chain

<h1> Geodesic HMC on Stiefel Manifolds </h1>

To implement Geodesic HMC on Stiefel Manifolds we need to define two operations.

1. Projection  
  This defines a projection onto the manifold itself from a position and a velocity.

2. Geodesic Flow  
  This defines the evolution of some time epsilon on the manifold from a starting position.

In [3]:
# The projection onto the stiefel manifold
# Input: 
#   V: Velocity
#   X: Position
def projection_Stiefel(V,X):
    inner = np.dot(X.T,V) + np.dot(V.T, X)
    return V - (1/2) * np.dot(X,inner)

# The geodesic flow of the particle on the Stiefel Manifold
# Input:
#   X: Position
#   V: Velocity
#   epsilon: The time-step.
def flow_Stiefel(X,V,epsilon):
    A = np.dot(X.T,V)
    S = np.dot(V.T,V)
    mexp = expm(-epsilon * A)
    t1 = expm(epsilon * np.bmat([[A, -S], [np.eye(A.shape[0]), A]]))
    t2 = np.bmat([[mexp, np.zeros(mexp.shape)],[np.zeros(mexp.shape), mexp]])
    F0 = np.bmat([X, V])
    R = np.dot(F0, np.dot(t1,t2))
    X_up, V_up = np.hsplit(R, 2)
    return np.array(X_up), np.array(V_up)

"""
Helper functions to transform angles to the Given's representation
and vice versa
"""
def left_rotate_counter_clockwise(A, angle, i, j):
    AR = A.copy()
    AR[i,:] = np.cos(angle)*A[i,:] - np.sin(angle)*A[j,:]
    AR[j,:] = np.sin(angle)*A[i,:] + np.cos(angle)*A[j,:]
    
    return AR

def right_rotate_counter_clockwise(A, angle, i, j):
    AR = A.copy()
    AR[:,i] = np.cos(angle)*A[:,i] + np.sin(angle)*A[:,j]
    AR[:,j] = -np.sin(angle)*A[:,i] + np.cos(angle)*A[:,j]
    
    return AR

def inverse_givens_transform(angles, n, p):
    G = np.eye(n)
    idx = 0
    for i in range(p):
        for j in range(i+1, n):
            G = right_rotate_counter_clockwise(G, angles[idx], i, j)
            idx = idx + 1
    return G[:,0:p]

def givens_transform(W):
    n, p = W.shape
    angles = [0 for _ in range(int(n*p-p*(p+1)/2))]
    idx = 0
    for i in range(p):
        for j in range(i+1, n):
            angle = np.arctan2(W[j,i],W[i,i])
            W = left_rotate_counter_clockwise(W, -angle, i, j)
            angles[idx] = angle
            idx = idx + 1
    
    return [a/np.pi for a in angles]

<h1> Testing PCA on the Manifold </h1>

Let's look at PCA on some generated data.

In [4]:
data = pd.read_csv('x.csv', index_col = 0)
data = data.values
N = data.shape[0]

<h1> Sampling via embedded manifold HMC </h1>

We will now try to perform the same technique by sampling the PPCA model.

In [5]:
# The model is y ~ N(Wz + \epsilon)
# This is the PPCA model and the log-likelihood is specified by Tipping and Bishop
# We build the potential energy = -log-likelihood and use autograd to find the derivative
sigma_hat = (1/N) * np.dot(data.T,data)
z = np.diag(np.array([1.,1.]))
def U(Q):
    C = agnp.dot(Q, agnp.dot(z,Q.T)) + np.eye(Q.shape[0])
    C_inv = agnp.linalg.inv(C)
    return N/2 * (agnp.log(agnp.linalg.det(C)) + agnp.trace(agnp.dot(C_inv,sigma_hat)))
    #return 1
grad_U = ag.grad(U)

In [6]:
# Setting up the params for the sampling
n_dim = 3
p_dim = 2
initial_params = np.eye(n_dim)[:,:p_dim]
n_iters = 5000
epsilon = 5e-2
L = 50
flow = flow_Stiefel
projection = projection_Stiefel
n_chains = 3

# Generate samples from the sampler

jobs = []
m = Manager()
return_dict = m.dict()
for i in range(n_chains):
    p = Process(target = Embedded_Manifold_HMC_MT, args = (initial_params, n_iters, epsilon, L, U, grad_U, flow, projection, i, return_dict))
    jobs.append(p)
    p.start()
    
for i in range(n_chains):
    p.join()

In [39]:
samples0 = return_dict[0] + return_dict[1] + return_dict[2] 
kept = samples0[5000:]
angle_rep = [givens_transform(i) for i in kept]
trace0 = go.Scatter(x = list(range(len(angle_rep))), y = [z[2] for z in angle_rep])
pyo.iplot([trace0])

In [40]:
hist0 = go.Histogram(x = [a[2] for a in angle_rep])

layout = go.Layout(
    xaxis=dict(
        title='# of samples'
    ),
    yaxis=dict(
        title='Angle'
    )
)

fig = go.Figure(data = [hist0], layout = layout)
pyo.iplot(fig)

In [42]:
pd.DataFrame(np.array(angle_rep)).to_csv('byrne_angles.csv', header = None, index = None)

In [91]:
a = PCA(n_components=3)
a.fit(data)
pca_score = a.explained_variance_ratio_
V = a.components_
x_pca_axis, y_pca_axis, z_pca_axis = V.T * pca_score / pca_score.min()

x_pca_axis, y_pca_axis, z_pca_axis = 3 * V.T
x_pca_plane = np.r_[x_pca_axis[:2], - x_pca_axis[1::-1]]
y_pca_plane = np.r_[y_pca_axis[:2], - y_pca_axis[1::-1]]
z_pca_plane = np.r_[z_pca_axis[:2], - z_pca_axis[1::-1]]

x_pca_plane.shape = (2, 2)
y_pca_plane.shape = (2, 2)
z_pca_plane.shape = (2, 2)


In [95]:
xx, yy = np.meshgrid(np.linspace(-2.25, 2.25, 4), np.linspace(-2.25, 2.25, 4))
zz = np.zeros(xx.shape)
z0 = np.ones(xx.shape)
for x, y, z in zip(xx, yy, zz):
    z2.append(a.transform(np.vstack([x,y,z]).T))

points = go.Scatter3d(x = data[:,0], y = data[:, 1], z = data[:,2], mode = 'markers')
true_surface = go.Surface(x = xx, y = yy , z = zz)
pca_surface = go.Surface(x = x_pca_plane, y = y_pca_plane, z = z_pca_plane)

pyo.iplot([points, true_surface, pca_surface])